# Data comparison by plotting

`line 3` *target_var_list* 입력

`line 7` *max_time_step* 입력

`line 9, 10` *dataset_1, dataset_2* 입력

`line 17` domain level 설정

`line 26` norm 스케일 조정

`line 28` region 설정 

In [ ]:
import xarray as xr
import numpy as np
import his_utils, his_plot
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from multiprocessing import Pool

dataset_1 = xr.open_dataset("/geodata2/S2S/DL/GC_output/2021-06-21/GC_11111111111_North America_scale_1.nc")
dataset_1 = his_utils.convert_scale(dataset_1).resample(time='1d').mean()

# dataset_2 = xr.open_dataset('/data/GC_output/2021-06-21/GC_10111111111_Southern Hemisphere Extra-tropics_wipeout_3.4028234663852886e+38.nc')
# dataset_2 = his_utils.convert_scale(dataset_2).resample(time='1d').mean()


In [ ]:
# 서로 다른 dataset별로 어떻게 생겨먹었는지 다 plot해서 뽑아버리기
# ['10m_u_component_of_wind',
#  '10m_v_component_of_wind',
#  '2m_temperature',
#  'geopotential_at_surface',
#  'land_sea_mask',
#  'mean_sea_level_pressure',
#  'total_precipitation_6hr',
#  'geopotential',
#  'specific_humidity',
#  'temperature',
#  'u_component_of_wind',
#  'v_component_of_wind',
#  'vertical_velocity',
#  'toa_incident_solar_radiation']


target_var_list = [ 'geopotential',
 'specific_humidity',
 'temperature',
 'u_component_of_wind',
 'v_component_of_wind',
 'vertical_velocity',
 '10m_u_component_of_wind',
 '10m_v_component_of_wind',
 '2m_temperature',
 'mean_sea_level_pressure',
 'total_precipitation_6hr']

max_time_step = 7

for var in target_var_list:
    diff = dataset_1[var]# - dataset_2[var]
    
    
    print("====================================")
    print(f"{var} std: {dataset_1[var].std().values}")
    print(f"{var} mean: {dataset_1[var].mean().values}")
    print("------------------------------------")
    #print(f"{var} std: {dataset_2[var].std().values}")
    #print(f"{var} mean: {dataset_2[var].mean().values}")
    #print("------------------------------------")
    # weights = np.cos(np.deg2rad(diff.lat))
    # weights.name = "weights"
    # weighted = diff.weighted(weights)
    mean_val = diff.mean(('lat', 'lon', 'time'))
    if 'level' in dataset_1[var].dims:
        std_all = diff.std(('lat', 'lon', 'time'))
        std_val = std_all.max().values
    else:
        std_val = diff.std(('lat', 'lon')).max().values
        norm = TwoSlopeNorm(vmin=mean_val - std_val, 
                            vcenter=mean_val, 
                            vmax=mean_val + std_val)

    # print(f"{var} std: {std_val}")
    # print(f"{var} mean: {mean_val}")

        
    
    

    title = "ERA5_perturbed"

    # level variables
    if 'level' in dataset_1[var].dims:
        mean_val = mean_val[0]
        std_all = std_all[0]
        arg_list = [(diff.isel(time=time_index, level=level_index), 
                     var, 
                     "platecarree", 
                     "RdBu_r", 
                     f'{var} {title} at t:{time_index}, level:{level_index}\n mean:{mean_val[level_index].values}, std:{std_all[level_index].values}', 
                     f'figure/{title} {var}_{time_index}_{level_index}.png',
                     None)
            for time_index in range(max_time_step)
            for level_index in range(len(diff.level))]
    
    # time-independent variables
    elif 'time' not in dataset_1[var].dims:
        arg_list = [(diff, 
                     var, 
                     "platecarree", 
                     "RdBu_r", 
                     f'{var} {title}', 
                     f'figure/{title} {var}.png',
                     norm)]
    
    # surface variables
    else:
        arg_list = [(diff.isel(time=time_index), 
                     var, 
                     "platecarree", 
                     "RdBu_r", 
                     f'{var} {title} at t:{time_index}', 
                     f'figure/{title} {var}_{time_index}.png',
                     norm)
            for time_index in range(max_time_step)]

    with Pool(processes=15) as pool:
        pool.map(his_plot.plot, arg_list)

geopotential std: nan
geopotential mean: nan
------------------------------------


/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan

mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [na

/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
If a partial dataset is provided, the normalization may not be accurate.
If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
If a partial dataset is provided, the normalization may not be accurate.If a partial dataset is provided, the normalization may not be accurate.

mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: n

...

mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
 If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nanWARNING: No normalization is provided. Using default normalization.
 If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nanWARNING: No normalization is provided. Using default normalization.
 If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan

If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], 

/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean: [nan], std: nan
If a partial dataset is provided, the normalization may not be accurate.
If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nanWARNING: No normalization is provided. Using default normalization.
 If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
If a partial dataset is provided, the normalization may not be accurate.
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
mean: [nan], std: nan
me

/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


ValueError: vmin, vcenter, vmax must increase monotonically

In [ ]:
# calculate weighted means & max value of each variables
ERA5 = xr.open_dataset("testdata/2022-01-01_tp.grib")
ERA5 = ERA5.stack(new_time=['time', 'step'])
ERA5 = ERA5.assign_coords(new_time=ERA5.valid_time.values)
ERA5 = ERA5.rename({'new_time': 'time'})

ERA5 = ERA5.drop_vars(['number', 'surface'])
tp = "tp"

weights = np.cos(np.deg2rad(ERA5.latitude))
weights.name = "weights"

mean_ERA = {}
mean_google = {}

max_ERA = {}
max_google = {}

for t in range(0, 14):
    mean_ERA[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('latitude', 'longitude')).values * 1000
    # mean_google[t]=ERA5[tp].isel(time=t).weighted(weights).mean(('lat', 'lon')).values

    max_ERA[t]=ERA5[tp].isel(time=t).max(('latitude', 'longitude')).values * 1000
    # max_google[t]=ERA5[tp].isel(time=t).max(('lat', 'lon')).values

# for var in ERA5.data_vars:
#     if var not in ['land_sea_mask', 'geopotential_at_surface']:
#         if 'level' in ERA5[var].dims:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).mean(dim='level').values
#         else:
#             mean_ERA[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
#             mean_google[var]=ERA5[var].isel(time=t).weighted(weights).mean(('lat', 'lon')).values
        
#     else:
#         mean_ERA[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values
#         mean_google[var]=ERA5[var].weighted(weights).mean(('lat', 'lon')).values